## Requirements

    !pip install requests_html
    !pip install wrapt --upgrade --ignore-installed
    !pip install tensorflow
    !pip3 install pandas numpy matplotlib yahoo_fin sklearn 

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from yahoo_fin import stock_info as si
from collections import deque

import os
import numpy as np
import pandas as pd
import random

Warning - Certain functionality 
             requires requests_html, which is not installed.
             
             Install using: 
             pip install requests_html
             
             After installation, you may have to restart your Python session.


In [3]:
# set seed, so we can get the same results after rerunning several times
np.random.seed(314)
tf.random.set_seed(314)
random.seed(314)

In [4]:
def shuffle_in_unison(a, b):
    # shuffle two arrays in the same way
    state = np.random.get_state()
    np.random.shuffle(a)
    np.random.set_state(state)
    np.random.shuffle(b)

def load_data(ticker, n_steps=50, scale=True, shuffle=True, lookup_step=1, split_by_date=True,
                test_size=0.2, feature_columns=['adjclose', 'volume', 'open', 'high', 'low']):
    """
    Loads data from Yahoo Finance source, as well as scaling, shuffling, normalizing and splitting.
    Params:
        ticker (str/pd.DataFrame): the ticker you want to load, examples include AAPL, TESL, etc.
        n_steps (int): the historical sequence length (i.e window size) used to predict, default is 50
        scale (bool): whether to scale prices from 0 to 1, default is True
        shuffle (bool): whether to shuffle the dataset (both training & testing), default is True
        lookup_step (int): the future lookup step to predict, default is 1 (e.g next day)
        split_by_date (bool): whether we split the dataset into training/testing by date, setting it 
            to False will split datasets in a random way
        test_size (float): ratio for test data, default is 0.2 (20% testing data)
        feature_columns (list): the list of features to use to feed into the model, default is everything grabbed from yahoo_fin
    """
    # see if ticker is already a loaded stock from yahoo finance
    if isinstance(ticker, str):
        # load it from yahoo_fin library
        df = si.get_data(ticker)
    elif isinstance(ticker, pd.DataFrame):
        # already loaded, use it directly
        df = ticker
    else:
        raise TypeError("ticker can be either a str or a `pd.DataFrame` instances")
    # this will contain all the elements we want to return from this function
    result = {}
    # we will also return the original dataframe itself
    result['df'] = df.copy()
    # make sure that the passed feature_columns exist in the dataframe
    for col in feature_columns:
        assert col in df.columns, f"'{col}' does not exist in the dataframe."
    # add date as a column
    if "date" not in df.columns:
        df["date"] = df.index
    if scale:
        column_scaler = {}
        # scale the data (prices) from 0 to 1
        for column in feature_columns:
            scaler = preprocessing.MinMaxScaler()
            df[column] = scaler.fit_transform(np.expand_dims(df[column].values, axis=1))
            column_scaler[column] = scaler
        # add the MinMaxScaler instances to the result returned
        result["column_scaler"] = column_scaler
    # add the target column (label) by shifting by `lookup_step`
    df['future'] = df['adjclose'].shift(-lookup_step)
    # last `lookup_step` columns contains NaN in future column
    # get them before droping NaNs
    last_sequence = np.array(df[feature_columns].tail(lookup_step))
    # drop NaNs
    df.dropna(inplace=True)
    sequence_data = []
    sequences = deque(maxlen=n_steps)
    for entry, target in zip(df[feature_columns + ["date"]].values, df['future'].values):
        sequences.append(entry)
        if len(sequences) == n_steps:
            sequence_data.append([np.array(sequences), target])
    # get the last sequence by appending the last `n_step` sequence with `lookup_step` sequence
    # for instance, if n_steps=50 and lookup_step=10, last_sequence should be of 60 (that is 50+10) length
    # this last_sequence will be used to predict future stock prices that are not available in the dataset
    last_sequence = list([s[:len(feature_columns)] for s in sequences]) + list(last_sequence)
    last_sequence = np.array(last_sequence).astype(np.float32)
    # add to result
    result['last_sequence'] = last_sequence
    # construct the X's and y's
    X, y = [], []
    for seq, target in sequence_data:
        X.append(seq)
        y.append(target)
    # convert to numpy arrays
    X = np.array(X)
    y = np.array(y)
    if split_by_date:
        # split the dataset into training & testing sets by date (not randomly splitting)
        train_samples = int((1 - test_size) * len(X))
        result["X_train"] = X[:train_samples]
        result["y_train"] = y[:train_samples]
        result["X_test"]  = X[train_samples:]
        result["y_test"]  = y[train_samples:]
        if shuffle:
            # shuffle the datasets for training (if shuffle parameter is set)
            shuffle_in_unison(result["X_train"], result["y_train"])
            shuffle_in_unison(result["X_test"], result["y_test"])
    else:    
        # split the dataset randomly
        result["X_train"], result["X_test"], result["y_train"], result["y_test"] = train_test_split(X, y, 
                                                                                test_size=test_size, shuffle=shuffle)
    # get the list of test set dates
    dates = result["X_test"][:, -1, -1]
    # retrieve test features from the original dataframe
    result["test_df"] = result["df"].loc[dates]
    # remove dates from the training/testing sets & convert to float32
    result["X_train"] = result["X_train"][:, :, :len(feature_columns)].astype(np.float32)
    result["X_test"] = result["X_test"][:, :, :len(feature_columns)].astype(np.float32)
    return result

In [5]:
def create_model(sequence_length, n_features, units=256, cell=LSTM, n_layers=2, dropout=0.3,
                loss="mean_absolute_error", optimizer="rmsprop", bidirectional=False):
    model = Sequential()
    for i in range(n_layers):
        if i == 0:
            # first layer
            if bidirectional:
                model.add(Bidirectional(cell(units, return_sequences=True), batch_input_shape=(None, sequence_length, n_features)))
            else:
                model.add(cell(units, return_sequences=True, batch_input_shape=(None, sequence_length, n_features)))
        elif i == n_layers - 1:
            # last layer
            if bidirectional:
                model.add(Bidirectional(cell(units, return_sequences=False)))
            else:
                model.add(cell(units, return_sequences=False))
        else:
            # hidden layers
            if bidirectional:
                model.add(Bidirectional(cell(units, return_sequences=True)))
            else:
                model.add(cell(units, return_sequences=True))
        # add dropout after each layer
        model.add(Dropout(dropout))
    model.add(Dense(1, activation="linear"))
    model.compile(loss=loss, metrics=["mean_absolute_error"], optimizer=optimizer)
    return model

In [6]:
import os
import time
from tensorflow.keras.layers import LSTM

# Window size or the sequence length
N_STEPS = 50
# Lookup step, 1 is the next day
LOOKUP_STEP = 15
# whether to scale feature columns & output price as well
SCALE = True
scale_str = f"sc-{int(SCALE)}"
# whether to shuffle the dataset
SHUFFLE = True
shuffle_str = f"sh-{int(SHUFFLE)}"
# whether to split the training/testing set by date
SPLIT_BY_DATE = False
split_by_date_str = f"sbd-{int(SPLIT_BY_DATE)}"
# test ratio size, 0.2 is 20%
TEST_SIZE = 0.2
# features to use
FEATURE_COLUMNS = ["adjclose", "volume", "open", "high", "low"]
# date now
date_now = time.strftime("%Y-%m-%d")
### model parameters
N_LAYERS = 2
# LSTM cell
CELL = LSTM
# 256 LSTM neurons
UNITS = 256
# 40% dropout
DROPOUT = 0.4
# whether to use bidirectional RNNs
BIDIRECTIONAL = False
### training parameters
# mean absolute error loss
# LOSS = "mae"
# huber loss
LOSS = "huber_loss"
OPTIMIZER = "adam"
BATCH_SIZE = 64
EPOCHS = 500
# Amazon stock market
ticker = "AMZN"
ticker_data_filename = os.path.join("data", f"{ticker}_{date_now}.csv")
# model name to save, making it as unique as possible based on parameters
model_name = f"{date_now}_{ticker}-{shuffle_str}-{scale_str}-{split_by_date_str}-\
{LOSS}-{OPTIMIZER}-{CELL.__name__}-seq-{N_STEPS}-step-{LOOKUP_STEP}-layers-{N_LAYERS}-units-{UNITS}"
if BIDIRECTIONAL:
    model_name += "-b"

In [7]:
# create these folders if they does not exist
if not os.path.isdir("results"):
    os.mkdir("results")
if not os.path.isdir("logs"):
    os.mkdir("logs")
if not os.path.isdir("data"):
    os.mkdir("data")

In [ ]:
# load the data
data = load_data(ticker, N_STEPS, scale=SCALE, split_by_date=SPLIT_BY_DATE, 
                shuffle=SHUFFLE, lookup_step=LOOKUP_STEP, test_size=TEST_SIZE, 
                feature_columns=FEATURE_COLUMNS)
# save the dataframe
data["df"].to_csv(ticker_data_filename)
# construct the model
model = create_model(N_STEPS, len(FEATURE_COLUMNS), loss=LOSS, units=UNITS, cell=CELL, n_layers=N_LAYERS,
                    dropout=DROPOUT, optimizer=OPTIMIZER, bidirectional=BIDIRECTIONAL)
# some tensorflow callbacks
checkpointer = ModelCheckpoint(os.path.join("results", model_name + ".h5"), save_weights_only=True, save_best_only=True, verbose=1)
tensorboard = TensorBoard(log_dir=os.path.join("logs", model_name))
# train the model and save the weights whenever we see 
# a new optimal model using ModelCheckpoint
history = model.fit(data["X_train"], data["y_train"],
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    validation_data=(data["X_test"], data["y_test"]),
                    callbacks=[checkpointer, tensorboard],
                    verbose=1)

Epoch 1/500
74/74 [==============================] - 28s 343ms/step - loss: 0.0030 - mean_absolute_error: 0.0322 - val_loss: 1.4261e-04 - val_mean_absolute_error: 0.0082

Epoch 00001: val_loss improved from inf to 0.00014, saving model to results\2020-12-29_AMZN-sh-1-sc-1-sbd-0-huber_loss-adam-LSTM-seq-50-step-15-layers-2-units-256.h5
Epoch 2/500
74/74 [==============================] - 22s 298ms/step - loss: 4.2193e-04 - mean_absolute_error: 0.0140 - val_loss: 1.1768e-04 - val_mean_absolute_error: 0.0084

Epoch 00002: val_loss improved from 0.00014 to 0.00012, saving model to results\2020-12-29_AMZN-sh-1-sc-1-sbd-0-huber_loss-adam-LSTM-seq-50-step-15-layers-2-units-256.h5
Epoch 3/500
74/74 [==============================] - 22s 296ms/step - loss: 4.2332e-04 - mean_absolute_error: 0.0142 - val_loss: 1.1796e-04 - val_mean_absolute_error: 0.0087

Epoch 00003: val_loss did not improve from 0.00012
Epoch 4/500
74/74 [==============================] - 22s 297ms/step - loss: 3.5684e-04 - mea


Epoch 00034: val_loss did not improve from 0.00011
Epoch 35/500
74/74 [==============================] - 23s 316ms/step - loss: 3.6089e-04 - mean_absolute_error: 0.0139 - val_loss: 1.0567e-04 - val_mean_absolute_error: 0.0069

Epoch 00035: val_loss improved from 0.00011 to 0.00011, saving model to results\2020-12-29_AMZN-sh-1-sc-1-sbd-0-huber_loss-adam-LSTM-seq-50-step-15-layers-2-units-256.h5
Epoch 36/500
74/74 [==============================] - 23s 311ms/step - loss: 2.7698e-04 - mean_absolute_error: 0.0124 - val_loss: 1.0401e-04 - val_mean_absolute_error: 0.0065

Epoch 00036: val_loss improved from 0.00011 to 0.00010, saving model to results\2020-12-29_AMZN-sh-1-sc-1-sbd-0-huber_loss-adam-LSTM-seq-50-step-15-layers-2-units-256.h5
Epoch 37/500
74/74 [==============================] - 24s 327ms/step - loss: 2.6188e-04 - mean_absolute_error: 0.0124 - val_loss: 1.0611e-04 - val_mean_absolute_error: 0.0066

Epoch 00037: val_loss did not improve from 0.00010
Epoch 38/500
74/74 [=========


Epoch 00068: val_loss did not improve from 0.00010
Epoch 69/500
74/74 [==============================] - 23s 318ms/step - loss: 2.6150e-04 - mean_absolute_error: 0.0127 - val_loss: 1.1142e-04 - val_mean_absolute_error: 0.0075

Epoch 00069: val_loss did not improve from 0.00010
Epoch 70/500
74/74 [==============================] - 23s 312ms/step - loss: 2.5952e-04 - mean_absolute_error: 0.0126 - val_loss: 1.5683e-04 - val_mean_absolute_error: 0.0093

Epoch 00070: val_loss did not improve from 0.00010
Epoch 71/500
74/74 [==============================] - 24s 326ms/step - loss: 2.5035e-04 - mean_absolute_error: 0.0125 - val_loss: 1.0321e-04 - val_mean_absolute_error: 0.0072

Epoch 00071: val_loss did not improve from 0.00010
Epoch 72/500
74/74 [==============================] - 24s 331ms/step - loss: 2.3551e-04 - mean_absolute_error: 0.0122 - val_loss: 1.4186e-04 - val_mean_absolute_error: 0.0087

Epoch 00072: val_loss did not improve from 0.00010
Epoch 73/500
74/74 [====================

Epoch 103/500
74/74 [==============================] - 23s 310ms/step - loss: 2.3475e-04 - mean_absolute_error: 0.0120 - val_loss: 1.5772e-04 - val_mean_absolute_error: 0.0083

Epoch 00103: val_loss did not improve from 0.00009
Epoch 104/500
74/74 [==============================] - 24s 322ms/step - loss: 2.2530e-04 - mean_absolute_error: 0.0122 - val_loss: 1.0032e-04 - val_mean_absolute_error: 0.0063

Epoch 00104: val_loss did not improve from 0.00009
Epoch 105/500
74/74 [==============================] - 24s 324ms/step - loss: 2.0673e-04 - mean_absolute_error: 0.0116 - val_loss: 1.4854e-04 - val_mean_absolute_error: 0.0076

Epoch 00105: val_loss did not improve from 0.00009
Epoch 106/500
74/74 [==============================] - 24s 328ms/step - loss: 2.3616e-04 - mean_absolute_error: 0.0126 - val_loss: 9.1694e-05 - val_mean_absolute_error: 0.0065

Epoch 00106: val_loss did not improve from 0.00009
Epoch 107/500
74/74 [==============================] - 22s 293ms/step - loss: 2.1954e-04

74/74 [==============================] - 23s 318ms/step - loss: 2.0690e-04 - mean_absolute_error: 0.0120 - val_loss: 1.1251e-04 - val_mean_absolute_error: 0.0075

Epoch 00139: val_loss did not improve from 0.00009
Epoch 140/500
74/74 [==============================] - 22s 303ms/step - loss: 2.0444e-04 - mean_absolute_error: 0.0115 - val_loss: 1.2789e-04 - val_mean_absolute_error: 0.0069

Epoch 00140: val_loss did not improve from 0.00009
Epoch 141/500
74/74 [==============================] - 23s 316ms/step - loss: 3.0158e-04 - mean_absolute_error: 0.0137 - val_loss: 9.0780e-05 - val_mean_absolute_error: 0.0063

Epoch 00141: val_loss did not improve from 0.00009
Epoch 142/500
74/74 [==============================] - 24s 325ms/step - loss: 2.2901e-04 - mean_absolute_error: 0.0119 - val_loss: 9.5187e-05 - val_mean_absolute_error: 0.0075

Epoch 00142: val_loss did not improve from 0.00009
Epoch 143/500
74/74 [==============================] - 24s 322ms/step - loss: 1.9633e-04 - mean_absolu

74/74 [==============================] - 24s 322ms/step - loss: 1.8741e-04 - mean_absolute_error: 0.0111 - val_loss: 1.1431e-04 - val_mean_absolute_error: 0.0074

Epoch 00175: val_loss did not improve from 0.00009
Epoch 176/500
74/74 [==============================] - 24s 326ms/step - loss: 1.8352e-04 - mean_absolute_error: 0.0108 - val_loss: 1.3218e-04 - val_mean_absolute_error: 0.0068

Epoch 00176: val_loss did not improve from 0.00009
Epoch 177/500
74/74 [==============================] - 23s 305ms/step - loss: 1.9986e-04 - mean_absolute_error: 0.0115 - val_loss: 9.6927e-05 - val_mean_absolute_error: 0.0062

Epoch 00177: val_loss did not improve from 0.00009
Epoch 178/500
74/74 [==============================] - 23s 316ms/step - loss: 1.8473e-04 - mean_absolute_error: 0.0110 - val_loss: 1.2228e-04 - val_mean_absolute_error: 0.0069

Epoch 00178: val_loss did not improve from 0.00009
Epoch 179/500
74/74 [==============================] - 24s 320ms/step - loss: 1.8880e-04 - mean_absolu

74/74 [==============================] - 23s 308ms/step - loss: 1.8207e-04 - mean_absolute_error: 0.0110 - val_loss: 9.3491e-05 - val_mean_absolute_error: 0.0062

Epoch 00211: val_loss did not improve from 0.00009
Epoch 212/500
74/74 [==============================] - 24s 321ms/step - loss: 1.7521e-04 - mean_absolute_error: 0.0108 - val_loss: 1.0913e-04 - val_mean_absolute_error: 0.0074

Epoch 00212: val_loss did not improve from 0.00009
Epoch 213/500
74/74 [==============================] - 24s 326ms/step - loss: 2.0535e-04 - mean_absolute_error: 0.0115 - val_loss: 9.3245e-05 - val_mean_absolute_error: 0.0064

Epoch 00213: val_loss did not improve from 0.00009
Epoch 214/500
74/74 [==============================] - 21s 289ms/step - loss: 1.8094e-04 - mean_absolute_error: 0.0106 - val_loss: 1.3956e-04 - val_mean_absolute_error: 0.0079

Epoch 00214: val_loss did not improve from 0.00009
Epoch 215/500
74/74 [==============================] - 22s 298ms/step - loss: 2.2137e-04 - mean_absolu

74/74 [==============================] - 24s 328ms/step - loss: 1.4405e-04 - mean_absolute_error: 0.0100 - val_loss: 9.7590e-05 - val_mean_absolute_error: 0.0062

Epoch 00279: val_loss did not improve from 0.00007
Epoch 280/500
74/74 [==============================] - 22s 294ms/step - loss: 1.6511e-04 - mean_absolute_error: 0.0103 - val_loss: 9.1288e-05 - val_mean_absolute_error: 0.0061

Epoch 00280: val_loss did not improve from 0.00007
Epoch 281/500
74/74 [==============================] - 24s 317ms/step - loss: 1.6193e-04 - mean_absolute_error: 0.0102 - val_loss: 9.0577e-05 - val_mean_absolute_error: 0.0066

Epoch 00281: val_loss did not improve from 0.00007
Epoch 282/500
74/74 [==============================] - 24s 323ms/step - loss: 1.5102e-04 - mean_absolute_error: 0.0101 - val_loss: 7.6559e-05 - val_mean_absolute_error: 0.0058

Epoch 00282: val_loss did not improve from 0.00007
Epoch 283/500
74/74 [==============================] - 24s 325ms/step - loss: 1.6308e-04 - mean_absolu

74/74 [==============================] - 22s 291ms/step - loss: 1.5091e-04 - mean_absolute_error: 0.0102 - val_loss: 5.3847e-05 - val_mean_absolute_error: 0.0057

Epoch 00348: val_loss did not improve from 0.00005
Epoch 349/500
74/74 [==============================] - 23s 311ms/step - loss: 1.4358e-04 - mean_absolute_error: 0.0098 - val_loss: 7.7784e-05 - val_mean_absolute_error: 0.0061

Epoch 00349: val_loss did not improve from 0.00005
Epoch 350/500
74/74 [==============================] - 20s 269ms/step - loss: 1.3116e-04 - mean_absolute_error: 0.0097 - val_loss: 6.2212e-05 - val_mean_absolute_error: 0.0056

Epoch 00350: val_loss did not improve from 0.00005
Epoch 351/500
74/74 [==============================] - 22s 301ms/step - loss: 1.6186e-04 - mean_absolute_error: 0.0103 - val_loss: 6.1404e-05 - val_mean_absolute_error: 0.0055

Epoch 00351: val_loss did not improve from 0.00005
Epoch 352/500
74/74 [==============================] - 21s 278ms/step - loss: 1.3779e-04 - mean_absolu

74/74 [==============================] - 22s 294ms/step - loss: 1.3083e-04 - mean_absolute_error: 0.0096 - val_loss: 7.1814e-05 - val_mean_absolute_error: 0.0069

Epoch 00382: val_loss did not improve from 0.00005
Epoch 383/500
74/74 [==============================] - 23s 310ms/step - loss: 1.5549e-04 - mean_absolute_error: 0.0102 - val_loss: 7.4848e-05 - val_mean_absolute_error: 0.0067

Epoch 00383: val_loss did not improve from 0.00005
Epoch 384/500
74/74 [==============================] - 22s 301ms/step - loss: 1.3118e-04 - mean_absolute_error: 0.0097 - val_loss: 5.5476e-05 - val_mean_absolute_error: 0.0055

Epoch 00384: val_loss did not improve from 0.00005
Epoch 385/500
74/74 [==============================] - 23s 313ms/step - loss: 1.2447e-04 - mean_absolute_error: 0.0094 - val_loss: 5.7458e-05 - val_mean_absolute_error: 0.0065

Epoch 00385: val_loss did not improve from 0.00005
Epoch 386/500
74/74 [==============================] - 24s 329ms/step - loss: 1.3600e-04 - mean_absolu

In [ ]:
import matplotlib.pyplot as plt

def plot_graph(test_df):
    """
    This function plots true close price along with predicted close price
    with blue and red colors respectively
    """
    plt.plot(test_df[f'true_adjclose_{LOOKUP_STEP}'], c='b')
    plt.plot(test_df[f'adjclose_{LOOKUP_STEP}'], c='r')
    plt.xlabel("Days")
    plt.ylabel("Price")
    plt.legend(["Actual Price", "Predicted Price"])
    plt.show()

In [ ]:
def get_final_df(model, data):
    """
    This function takes the `model` and `data` dict to 
    construct a final dataframe that includes the features along 
    with true and predicted prices of the testing dataset
    """
    # if predicted future price is higher than the current, 
    # then calculate the true future price minus the current price, to get the buy profit
    buy_profit  = lambda current, true_future, pred_future: true_future - current if pred_future > current else 0
    # if the predicted future price is lower than the current price,
    # then subtract the true future price from the current price
    sell_profit = lambda current, true_future, pred_future: current - true_future if pred_future < current else 0
    X_test = data["X_test"]
    y_test = data["y_test"]
    # perform prediction and get prices
    y_pred = model.predict(X_test)
    if SCALE:
        y_test = np.squeeze(data["column_scaler"]["adjclose"].inverse_transform(np.expand_dims(y_test, axis=0)))
        y_pred = np.squeeze(data["column_scaler"]["adjclose"].inverse_transform(y_pred))
    test_df = data["test_df"]
    # add predicted future prices to the dataframe
    test_df[f"adjclose_{LOOKUP_STEP}"] = y_pred
    # add true future prices to the dataframe
    test_df[f"true_adjclose_{LOOKUP_STEP}"] = y_test
    # sort the dataframe by date
    test_df.sort_index(inplace=True)
    final_df = test_df
    # add the buy profit column
    final_df["buy_profit"] = list(map(buy_profit, 
                                    final_df["adjclose"], 
                                    final_df[f"adjclose_{LOOKUP_STEP}"], 
                                    final_df[f"true_adjclose_{LOOKUP_STEP}"])
                                    # since we don't have profit for last sequence, add 0's
                                    )
    # add the sell profit column
    final_df["sell_profit"] = list(map(sell_profit, 
                                    final_df["adjclose"], 
                                    final_df[f"adjclose_{LOOKUP_STEP}"], 
                                    final_df[f"true_adjclose_{LOOKUP_STEP}"])
                                    # since we don't have profit for last sequence, add 0's
                                    )
    return final_df

In [ ]:
def predict(model, data):
    # retrieve the last sequence from data
    last_sequence = data["last_sequence"][-N_STEPS:]
    # expand dimension
    last_sequence = np.expand_dims(last_sequence, axis=0)
    # get the prediction (scaled from 0 to 1)
    prediction = model.predict(last_sequence)
    # get the price (by inverting the scaling)
    if SCALE:
        predicted_price = data["column_scaler"]["adjclose"].inverse_transform(prediction)[0][0]
    else:
        predicted_price = prediction[0][0]
    return predicted_price

In [ ]:
# load optimal model weights from results folder
model_path = os.path.join("results", model_name) + ".h5"
model.load_weights(model_path)

In [ ]:
# evaluate the model
loss, mae = model.evaluate(data["X_test"], data["y_test"], verbose=0)
# calculate the mean absolute error (inverse scaling)
if SCALE:
    mean_absolute_error = data["column_scaler"]["adjclose"].inverse_transform([[mae]])[0][0]
else:
    mean_absolute_error = mae

In [ ]:
# get the final dataframe for the testing set
final_df = get_final_df(model, data)

In [ ]:
# predict the future price
future_price = predict(model, data)

In [ ]:
# we calculate the accuracy by counting the number of positive profits
accuracy_score = (len(final_df[final_df['sell_profit'] > 0]) + len(final_df[final_df['buy_profit'] > 0])) / len(final_df)
# calculating total buy & sell profit
total_buy_profit  = final_df["buy_profit"].sum()
total_sell_profit = final_df["sell_profit"].sum()
# total profit by adding sell & buy together
total_profit = total_buy_profit + total_sell_profit
# dividing total profit by number of testing samples (number of trades)
profit_per_trade = total_profit / len(final_df)

In [ ]:
# printing metrics
print(f"Future price after {LOOKUP_STEP} days is {future_price:.2f}$")
print(f"{LOSS} loss:", loss)
print("Mean Absolute Error:", mean_absolute_error)
print("Accuracy score:", accuracy_score)
print("Total buy profit:", total_buy_profit)
print("Total sell profit:", total_sell_profit)
print("Total profit:", total_profit)
print("Profit per trade:", profit_per_trade)

In [ ]:
# plot true/pred prices graph
plot_graph(final_df)

In [ ]:
print(final_df.tail(10))
# save the final dataframe to csv-results folder
csv_results_folder = "csv-results"
if not os.path.isdir(csv_results_folder):
    os.mkdir(csv_results_folder)
csv_filename = os.path.join(csv_results_folder, model_name + ".csv")
final_df.to_csv(csv_filename)